# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/cf72033452b01be31d061c5e1a6a67b550983a5b53df44866f4b108c.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **New York, New York, United States**, and the stations the data comes from are shown on the map below.

In [34]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'cf72033452b01be31d061c5e1a6a67b550983a5b53df44866f4b108c')

In [81]:
def question_one():

    df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/cf72033452b01be31d061c5e1a6a67b550983a5b53df44866f4b108c.csv')
    
    # divide Data_Value column by 10
    df['Data_Value'] *= .1
    
    # convert date column to date format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # get date, month, and year columns from date column
    df['day'] = pd.DatetimeIndex(df['Date']).day
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['year'] = pd.DatetimeIndex(df['Date']).year
    
    #set new month-day index
    df.set_index(['month', 'day'], inplace=True)
    
    # remove leap-year rows for Feb 29 
    df = df[~df.index.isin(df.loc[2,29].index)]
    
    # split df into two periods
    df_2015 = df[df['year'] == 2015]
    df = df[df['year'] <= 2014]
    
    # split df into max and min values based on value in 'Element'
    df_max = df[df['Element'] == 'TMAX']
    max_vals = df_max.groupby(level=['month', 'day'])['Data_Value'].agg({'max_temp': max})
    df_min = df[df['Element'] == 'TMIN']
    min_vals = df_min.groupby(level=['month', 'day'])['Data_Value'].agg({'min_temp': min})
    
    # repeat same process for 2015 data
    df_2015_max = df_2015[df_2015['Element'] == 'TMAX']
    max_2015_vals = df_2015_max.groupby(level=['month', 'day'])['Data_Value'].agg({'max_temp': max})
    df_2015_min = df_2015[df_2015['Element'] == 'TMIN']
    min_2015_vals = df_2015_min.groupby(level=['month', 'day'])['Data_Value'].agg({'min_temp': min})
    
    
    # plotting
    plt.figure(figsize=(12,8))
    
    # plot max and min temp line graphs for 2005-2014 data
    plt.plot(list(min_vals['min_temp']), alpha=.5, color='darkturquoise')
    plt.plot(list(max_vals['max_temp']), alpha =.6, color='darkorange')
    
    # set new x values for chart to months, spaced out by days per month
    plt.xlim([0,365])
    months = ['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sep','Oct','Nov','Dec']
    month_interval = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    plt.xticks(month_interval, months)
    
    # fill data between min and max values for 2005-2014 data
    plt.gca().fill_between(range(len(max_vals['max_temp'])),
                           min_vals['min_temp'], max_vals['max_temp'],
                           color='lightslategrey',
                           alpha=.1)
    

    # Implementation of step two. Filter 2015 values to plot only those greater than
    # max values from 2005-2014 and those less than the min values from 2005-2014
    plt.plot(list(min_2015_vals[min_2015_vals < min_vals]['min_temp']), 'o', color='blue', alpha=.6)
    plt.plot(list(max_2015_vals[max_2015_vals > max_vals]['max_temp']), 'o', color='red', alpha=.6)
    
    
    # create legend centered at the bottom of the chart
    plt.legend(['2005-2014 Lows','2005-2014 Highs','2015 Broken Min','2015 Broken Max'],
               loc='lower center', frameon=False, bbox_to_anchor=(0.55, .05), ncol=2)
    
    
    # set titles and axis labels
    plt.title('Record Temperatures for 2005-2014 \n with 2015 Overlay near New York', fontsize=15, alpha=.8)
    plt.ylabel('Temperature $(C^{\circ}$)', size=12)
    plt.xlabel('Month', size=12)
        
    # remove frame from graph
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
        
    return(plt.show())

     
question_one()    

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
